# Bloomberg BQuant Spotlight Webinar Series: High Yield Interest Coverage and OAS

This is a companion notebook to the [Bloomberg High Yield Interest Coverage and OAS](https://blinks.bloomberg.com/screens/PLYR%20VOD%20332757057) webinar video.

There has been a surge in government borrowing over the past 10 years fueled by suppressed interest rates, which has led to increased corporate borrowing. Now companies are faced with the challenge of navigating a higher expected rate environment which will impact their borrowing capability and earnings. This notebook is an analysis of the High Yield Issuers' financial health indicated by Interest Coverage. It will first illustrate how to create two aggregate interest coverage trend charts, and then plot particular bonds that may seem attractive in the context of Interest Coverage and OAS.
> <span style="color:orange">Note: Please allow several minutes for the calculation in Part 1.</span>


### Part 1

In [ ]:
# import libraries
import bql
import datetime as dt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import bqviz as bqv
from bqwidgets import DataGrid
from ipywidgets import VBox,HBox,Button,Layout

bq=bql.Service()

In [ ]:
# BQL functions to use when looping through date range
def bq_create_fundamentals_univ_from_index(index_str,as_of_date=dt.datetime.strftime(dt.datetime.now(),'%Y-%m-%d')):
    """
        Summary:    This function produces a list of members expected to store the financials for an entity, removing duplicate tickers 
                    in cases where two securities have the same company filing (i.e. DISCA and DISCK)
                    This also excludes financial companies using bics sectors
        Returns:    bql universe item

    """
    univ=bq.univ.translatesymbols(bq.univ.filter(bq.univ.members(index_str,DATES=as_of_date),
                                                bq.func.notequals(bq.data.classification_name('bics','1'),'Financials')),
                                  'CORP','FUNDAMENTALTICKER')
    unique_set=set(bql.combined_df(bq.execute(bql.Request(univ,bq.data.id()))).index)
    unique_set.discard('NULL_ID')
    
    univ_edit=bq.univ.list(unique_set)

    return univ_edit

def bq_avg_oas_index_request(index_str,as_of_date=dt.datetime.strftime(dt.datetime.now(),'%Y-%m-%d'),spread_period_str='-1m',spread_period_frq='d',pricing_source='BVAL'):
    """
        Summary:    This function takes a fixed income index and builds a request that represents the avg OAS for a specified time
                    period for the group
                    This also excludes financial companies using bics sectors
        Returns:    bql request

    """
    univ=bq.univ.filter(bq.univ.members(index_str,DATES=as_of_date),
                        bq.func.notequals(bq.data.classification_name('bics','1'),'Financials'))
    
    avg_oas=bq.func.avg(bq.func.dropna(bq.data.spread(dates=bq.func.range(START=spread_period_str,END='-0d',FRQ=spread_period_frq),spread_type='OAS',pricing_source=pricing_source)))['value']
    group_avg_oas=bq.func.avg(bq.func.group(bq.func.applypreferences(avg_oas,ANCHORRELDATESBY='BROADCAST')))
    
    req=bql.Request(univ,{'Avg OAS':group_avg_oas},with_params={'mode':'cached'})

    return req


def bq_build_agg_ratio_metric(numerator_str,denominator_str):
    """
        Summary:	This function creates a BQL item, given two inputs (numerator and denominator) 
                    The BQL item represents sum of members' adjusted numerator divided by sum of the adjusted denominator.
                    Denominator is adjusted for numerator data availability and vice versa.
        Returns:    bql data item
    """
    bq_num_params={'CURRENCY':'USD',
                  'FA_PERIOD_TYPE':'LTM',
                  'FA_FILING_STATUS':'MRXP'}
    bq_denom_params={'CURRENCY':'USD',
                  'FA_PERIOD_TYPE':'LTM',
                  'FA_FILING_STATUS':'MRXP'}
    bq_numerator=eval("bq.data."+numerator_str+"(**bq_num_params)")
    bq_denominator=eval("bq.data."+denominator_str+"(**bq_denom_params)")
    bq_numerator=bq.func.if_(bq.func.or_(bq.func.equals(bq_numerator,bql.NA),bq.func.equals(bq_denominator,bql.NA)),
                                bql.NA,
                                bq_numerator)
    bq_denominator=bq.func.if_(bq.func.or_(bq.func.equals(bq_numerator,bql.NA),bq.func.equals(bq_denominator,bql.NA)),
                                bql.NA,
                                bq_denominator)
    bq_agg_metric=bq.func.sum(bq.func.group(bq_numerator['value']))/bq.func.sum(bq.func.group(bq_denominator['value']))

    return bq_agg_metric

def bq_build_median_ratio_metric(numerator_str,denominator_str):
    """
        Summary:	This function creates a BQL item, given two inputs (numerator and denominator) 
                    The BQL item represents median of members' adjusted numerator divided by sum of the adjusted denominator.
                    Denominator is adjusted for numerator data availability and vice versa.
                    If numerator or denominator is negative, the ratio is not calculated.
        Returns:    bql data item
    """
    bq_num_params={'CURRENCY':'USD',
                  'FA_PERIOD_TYPE':'LTM',
                  'FA_FILING_STATUS':'MRXP'}
    bq_denom_params={'CURRENCY':'USD',
                  'FA_PERIOD_TYPE':'LTM',
                  'FA_FILING_STATUS':'MRXP'}
    bq_numerator=eval("bq.data."+numerator_str+"(**bq_num_params)")
    bq_denominator=eval("bq.data."+denominator_str+"(**bq_denom_params)")
    bq_numerator=bq.func.if_(bq.func.or_(bq.func.equals(bq_numerator,bql.NA),bq.func.equals(bq_denominator,bql.NA)),
                                bql.NA,
                                bq_numerator)
    bq_denominator=bq.func.if_(bq.func.or_(bq.func.equals(bq_numerator,bql.NA),bq.func.equals(bq_denominator,bql.NA)),
                                bql.NA,
                                bq_denominator)
    
    bq_metric=bq.func.if_(bq.func.or_(bq.func.lessthanorequals(bq_numerator,0),bq.func.lessthanorequals(bq_denominator,0)),bql.NA,bq_numerator['value']/bq_denominator['value'])
    
    bq_group_median=bq.func.median(bq.func.group(bq_metric))

    return bq_group_median

def bq_build_ratio_metric(numerator_str,denominator_str):
    """
        Summary:	This function creates a BQL item, given two inputs (numerator and denominator) 
                    The BQL item represents a ratio with an adjusted numerator divided by the adjusted denominator.
                    Denominator is adjusted for numerator data availability and vice versa.
                    If numerator or denominator is negative, the ratio is not calculated.
        Returns:    bql data item
    """
    bq_num_params={'CURRENCY':'USD',
                  'FA_PERIOD_TYPE':'LTM',
                  'FA_FILING_STATUS':'MRXP'}
    bq_denom_params={'CURRENCY':'USD',
                  'FA_PERIOD_TYPE':'LTM',
                  'FA_FILING_STATUS':'MRXP'}
    bq_numerator=eval("bq.data."+numerator_str+"(**bq_num_params)")
    bq_denominator=eval("bq.data."+denominator_str+"(**bq_denom_params)")
    bq_numerator=bq.func.if_(bq.func.or_(bq.func.equals(bq_numerator,bql.NA),bq.func.equals(bq_denominator,bql.NA)),
                                bql.NA,
                                bq_numerator)
    bq_denominator=bq.func.if_(bq.func.or_(bq.func.equals(bq_numerator,bql.NA),bq.func.equals(bq_denominator,bql.NA)),
                                bql.NA,
                                bq_denominator)
    
    bq_metric=bq.func.if_(bq.func.or_(bq.func.lessthanorequals(bq_numerator,0),bq.func.lessthanorequals(bq_denominator,0)),bql.NA,bq_numerator['value']/bq_denominator['value'])
    
    return bq_metric

def create_line_chart_fig(x_axis,primary_y_list,primary_y_names_list,primary_y_data_label=None,primary_y_last_point_text=True,secondary_y_list=None,secondary_y_data_label=None,secondary_y_last_point_text=False,chart_title=None):
        """
        Summary: This function returns a matplotlib figure that is a line chart. Easy function to chart 2 Y-axis 
        Returns: matplotlib fig for line chart
        """
        my_palette=['blue', 'green','purple','gold','grey','magenta','orange','red','brown','chartreuse']
        
        #setup/validation
        if type(primary_y_list)!=list:
            primary_y_list=[primary_y_list]
        if type(primary_y_names_list)!=list:
            primary_y_names_list=[primary_y_names_list]
        
        if secondary_y_list is not None:
            if type(secondary_y_list)!=list:
                secondary_y_list=[secondary_y_list]
            secondary_y_list=secondary_y_list[:1]
        primary_y_list=primary_y_list[:len(my_palette)]
        primary_y_names_list=primary_y_names_list[:len(my_palette)]

        fig, ax1 = plt.subplots()
        
        #primary y-axis (one line vs. multi-line)
        if len(primary_y_list)==1:
            
            ax1.plot(x_axis, primary_y_list[0], my_palette[0])
            ax1.set_xlabel('Date')
            if primary_y_data_label is None:
                ax1.set_ylabel(primary_y_list[0].name, color=my_palette[0])
            else:
                ax1.set_ylabel(primary_y_data_label, color=my_palette[0])
            if primary_y_last_point_text==True:
                style = dict(size=10, color=my_palette[0])
                ax1.text(x_axis[-1],primary_y_list[0][-1],str(np.around(primary_y_list[0][-1],decimals=4)),**style)
            ax1.tick_params('y', colors=my_palette[0])
            
            if chart_title is None:
                plt.title(str(primary_y_names_list[0]))
        else:
            
            num=0
            
            for y in primary_y_list:
                if len(x_axis)==len(y):
                    ax1.plot(x_axis, y, marker='', color=my_palette[num], linewidth=1, alpha=0.9, label=primary_y_names_list[num])
                    if primary_y_last_point_text==True:
                        style = dict(size=10, color=my_palette[num])
                        ax1.text(x_axis[-1],y[-1],str(np.around(y[-1],decimals=4)),**style)
                    num+=1

            labs = [l.get_label() for l in ax1.lines]
            
            leg=fig.legend(ax1.lines, labs, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                           ncol=2, mode="expand", borderaxespad=0.)

            
            ax1.set_ylabel(primary_y_data_label, color='k')
            
            labs = [l.get_label() for l in ax1.lines]
            leg=fig.legend(ax1.lines, labs, bbox_to_anchor=(0., 1.02, 1., .102), loc=3,ncol=2, mode="expand", borderaxespad=0.)
            leg.get_title().set_fontsize('12')
            for text in leg.get_texts():
                text.set_color('k')
            frame = leg.get_frame()
            frame.set_edgecolor('xkcd:light grey')
            frame.set_facecolor('xkcd:light grey')

            if chart_title is None:
                plt.title(str(primary_y_data_label))
        
        #scondary y-axis (one line vs. multi-line)
        if secondary_y_list is not None:
            ax2 = ax1.twinx()
            
            if len(secondary_y_list)>1:
                for y in secondary_y_list:
                    if len(x_axis)==len(y):
                        ax2.plot(x_axis, y, marker='', linestyle='-',color=my_palette(num), linewidth=1, alpha=0.9, label=primary_y_names_list[num])
                        if secondary_y_last_point_text==True:
                            style = dict(size=10, color=my_palette[num])
                            ax2.text(x_axis[-1],y[-1],str(np.around(y[-1],decimals=4)),**style)
                        num+=1
            else:
                ax2.plot(x_axis, secondary_y_list[0], 'r')
                ax2.set_ylabel(secondary_y_data_label, color='r')
                ax2.tick_params('y', colors='r')
                if secondary_y_last_point_text==True:
                    style = dict(size=10, color='r')
                    ax2.text(x_axis[-1],secondary_y_list[0][-1],str(np.around(secondary_y_list[0][-1],decimals=4)),**style)

        if chart_title is not None:
            plt.title(str(chart_title))

        #display requirements for a chart
        fig.patch.set_facecolor('xkcd:light grey')
        fig.set_size_inches(10, 5)
        fig.tight_layout()
        ax1.grid(color = 'PowderBlue', linewidth=1.25, linestyle="--")
        plt.close('all')
        return fig

In [ ]:
date_range=pd.date_range(start=dt.datetime(dt.datetime.today().year - 8, 12, 31),end=dt.datetime.today(),freq='Q')
bq_date_str_range_list=[dt.datetime.strftime(date,'%Y-%m-%d') for date in date_range]
fi_indices_d={'US Corp HY':'LF98TRUU Index'}

In [ ]:
bq_group_interest_coverage=bq_build_agg_ratio_metric('ebitda','is_int_expense')
bq_group_interest_coverage_d={'Interest Coverage Aggregate':bq_group_interest_coverage['value']}
index_agg_requests_d={}
multi_agg_index_df=pd.DataFrame(index=date_range)

bq_median_interest_coverage=bq_build_median_ratio_metric('ebitda','is_int_expense')
bq_median_interest_coverage_d={'Interest Coverage Median':bq_median_interest_coverage['value']}
index_median_requests_d={}
multi_median_index_df=pd.DataFrame(index=date_range)

oas_avg_request_d={}
multi_oas_avg_index_df=pd.DataFrame(index=date_range)

# Build Requests for Agg & Medians & OAS
for name,index_name in fi_indices_d.items():
    agg_request_per_date_list=[]
    medians_request_per_date_list=[]
    oas_avg_request_per_date_list=[]
    
    for date in bq_date_str_range_list:
        # Aggs Request
        req_agg=bql.Request(bq_create_fundamentals_univ_from_index(index_name,as_of_date=date), # universe
                        bq_group_interest_coverage_d, #field
                        with_params={'dates':date,'mode':'cached'})
                      
        agg_request_per_date_list.append(req_agg)
        
        # Medians Request
        req_median=bql.Request(bq_create_fundamentals_univ_from_index(index_name,as_of_date=date), # universe
                        bq_median_interest_coverage_d, #field
                        with_params={'dates':date,'mode':'cached'})
                           
        medians_request_per_date_list.append(req_median)
        
        # Avg OAS Request
        req_oas=bq_avg_oas_index_request(index_name,as_of_date=date,spread_period_str='-1m',spread_period_frq='d',pricing_source='BVAL')
        oas_avg_request_per_date_list.append(req_oas)
        

    index_agg_requests_d[name]=agg_request_per_date_list
    index_median_requests_d[name]=medians_request_per_date_list
    oas_avg_request_d[name]=oas_avg_request_per_date_list

# Request and Store Agg Requests
for name,list_req in index_agg_requests_d.items():
    counter=0
    execute_subset=list(bq.execute_many(index_agg_requests_d[name]))
    for date in date_range:
        multi_agg_index_df.loc[date,name]=execute_subset[counter][0].df()['Interest Coverage Aggregate'][0]
        counter+=1

        
# Request and Store Median Requests
for name,list_req in index_median_requests_d.items():
    counter=0
    execute_subset=list(bq.execute_many(index_median_requests_d[name]))
    for date in date_range:
        multi_median_index_df.loc[date,name]=execute_subset[counter][0].df()['Interest Coverage Median'][0]
        counter+=1

# Request and Store OAS Requests
for name,list_req in oas_avg_request_d.items():
    counter=0
    execute_subset=list(bq.execute_many(oas_avg_request_d[name]))
    for date in date_range:
        multi_oas_avg_index_df.loc[date,name]=execute_subset[counter][0].df()['Avg OAS'][0]
        counter+=1        

In [ ]:
agg_chart=create_line_chart_fig(list(multi_agg_index_df.index),multi_agg_index_df['US Corp HY'],['Aggregate Interest Coverage'],secondary_y_list=multi_oas_avg_index_df['US Corp HY'],secondary_y_data_label='OAS',secondary_y_last_point_text=True)
agg_chart

In [ ]:
median_chart=create_line_chart_fig(list(multi_median_index_df.index),multi_median_index_df['US Corp HY'],['Median Interest Coverage'],secondary_y_list=multi_oas_avg_index_df['US Corp HY'],secondary_y_data_label='OAS',secondary_y_last_point_text=True)
median_chart

### Part 2

In [ ]:
bond_univ=bq.univ.members('LF98TRUU Index')
company_univ=bq_create_fundamentals_univ_from_index('LF98TRUU Index')

bq_simplified_rating=bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),3),"AAA"),"AAA",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),2),"AA"),"AA",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),1),"A"),"A",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),3),"BBB"),"BBB",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),2),"BB"),"BB",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),1),"B"),"B",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),3),"CCC"),"CCC",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),2),"CC"),"CC",
                                bq.func.if_(bq.func.equals(bq.func.left(bq.data.bb_composite(),1),"C"),"C","Other")))))))))
bq_interest_coverage_ticker=bq.data.fundamental_ticker()

bq_bval_oas_spread_series=bq.func.dropna(bq.data.spread(spread_type='OAS',dates=bq.func.range('-6m','-0d')))
bq_bval_oas_spread_count=bq.func.count(bq_bval_oas_spread_series)
bq_bval_oas_current_z=bq.func.if_(bq_bval_oas_spread_count==0,bql.NA,bq.func.last(bq.func.zscore(bq.func.group(bq.func.dropna(bq_bval_oas_spread_series),bq.data.id()))))['value']

bond_analysis_d={'Simple Rating':bq_simplified_rating,
               'Ticker':bq_interest_coverage_ticker,
                'Industry':bq.data.classification_name('bics','2'),
                'OAS ZScore (vs 6M)':bq_bval_oas_current_z}



company_analysis_d={'Interest Coverage':bq_build_ratio_metric('ebitda','is_int_expense')}


req_bond=bql.Request(bq.univ.members('LF98TRUU Index'),bond_analysis_d,with_params={'mode':'cached'})
bqexec_bond=bq.execute(req_bond)

req_company=bql.Request(company_univ,company_analysis_d)
bqexec_company=bq.execute(req_company)

bond_df=bql.combined_df(bqexec_bond)
company_df=bql.combined_df(bqexec_company)

bond_df['Interest Coverage']=[company_df.loc[ticker]['Interest Coverage'] if ticker in company_df.index else np.NaN for ticker in bond_df['Ticker']]
bond_df=bond_df.loc[pd.isnull(bond_df['Interest Coverage'])==False]

In [ ]:
upperbound=company_df.quantile(0.95)['Interest Coverage']
lowerbound=company_df.quantile(0.05)['Interest Coverage']
bond_df['Interest Coverage Adj']= [max(lowerbound, min(x, upperbound)) for x in bond_df['Interest Coverage']]

industries_with_enough_data=list(bond_df.groupby(['Industry'])['Ticker'].count()[bond_df.groupby(['Industry'])['Ticker'].count()>5].index)
bond_df_scatter=bond_df.loc[bond_df['Industry'].isin(industries_with_enough_data)].copy()
bond_df_scatter['Industry Interest Coverage Mean']=[bond_df_scatter.groupby(['Industry'])['Interest Coverage Adj'].mean().loc[ind] for ind in bond_df_scatter['Industry']]
bond_df_scatter['Industry Relative Interest Coverage']=bond_df_scatter['Interest Coverage Adj']/bond_df_scatter['Industry Interest Coverage Mean']
bond_df_scatter.head()

In [ ]:
df=bond_df_scatter.loc[:,['OAS ZScore (vs 6M)','Industry Relative Interest Coverage']]
int_scatter = bqv.InteractiveScatterPlot(df, hide_controls=False, 
                                         reg_line=False, indexes=False)
int_scatter.x_control.value='OAS ZScore (vs 6M)'
int_scatter.y_control.value='Industry Relative Interest Coverage'

select_data_button=Button(description='Show Selection',layout=Layout(width='120px',left='89px'))
select_data_button.style.button_color='green'

column_defs=[
            {'headerName': 'Corp Ticker', 'field': 'Corp Ticker', 'width': 120, 'pinned': 'left'},
            {'headerName': 'Industry', 'field': 'Industry', 'width': 200, 'pinned': 'left'},
            {'headerName': 'Rating', 'field': 'Simple Rating', 'width': 120},
            {'headerName': 'OAS', 'field': 'OAS ZScore (vs 6M)', 'width': 150},
            {'headerName': 'Equity Ticker', 'field': 'Ticker', 'width': 120},
            {'headerName': 'Interest Coverage Adj', 'field': 'Interest Coverage Adj', 'width': 180},
            {'headerName': 'Industry Relative IC', 'field': 'Industry Relative Interest Coverage', 'width': 180},
            ]
dg=DataGrid(data=[],column_defs=column_defs,layout=Layout(width='840px',height='450px'))


def print_scatter_selection(clk):
    dg.data=bond_df_scatter.filter(items=list(int_scatter.selected_data.index),axis=0).reset_index().rename(columns={'ID':'Corp Ticker'}).round(decimals=2)
    
select_data_button.on_click(print_scatter_selection)
display_box=HBox([VBox([int_scatter.show(),select_data_button]),dg])

display_box